In [1]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

seed = 2023

In [2]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search,
)

In [3]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-08-18 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-18 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-18 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-18 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-18 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# SWNU Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [7]:
features = ["timeline_index"]
standardise_method = [None]
num_features = len(features)
add_time_in_path = True

In [8]:
num_epochs = 100
embedding_dim = 384
dimensions = [15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[64, 64], [128, 128], [256, 256], [512, 512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [9]:
size = 20
(
    swnu_network_umap_kfold_20,
    best_swnu_network_umap_kfold_20,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    embedding_dim=embedding_dim,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features,
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_20_kfold.csv


In [10]:
swnu_network_umap_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2692122/1182452915.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_20.groupby(["dimensions",


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (64, 64)       0.1          0.0001         0.627522   
                                                       0.0005         0.625862   
                                                       0.0010         0.621456   
                                          0.5          0.0001         0.629438   
                                                       0.0005         0.630077   
                                                       0.0010         0.646169   
                           (128, 128)     0.1          0.0001         0.620434   
                                                       0.0005         0.622861   
                                                       0.0010         0.623308   
                                          0.5          0.0001         0.619221   
                                                       0.0005         0.621392   
                                                       0.0010         0.641379   
                           (256, 256)     0.1          0.0001         0.632886   
                                                       0.0005         0.625926   
                                                       0.0010         0.619029   
                                          0.5          0.0001         0.638059   
                                                       0.0005         0.617114   
                                                       0.0010         0.638697   
                           (512, 512)     0.1          0.0001         0.624393   
                                                       0.0005         0.618199   
                                                       0.0010         0.611558   
                                          0.5          0.0001         0.623946   
                                                       0.0005         0.624968   
                                                       0.0010         0.633461   
           (12,)           (64, 64)       0.1          0.0001         0.620881   
                                                       0.0005         0.613793   
                                                       0.0010         0.630268   
                                          0.5          0.0001         0.638953   
                                                       0.0005         0.633780   
                                                       0.0010         0.625926   
                           (128, 128)     0.1          0.0001         0.627586   
                                                       0.0005         0.626054   
                                                       0.0010         0.627458   
                                          0.5          0.0001         0.631545   
                                                       0.0005         0.625479   
                                                       0.0010         0.624393   
                           (256, 256)     0.1          0.0001         0.618774   
                                                       0.0005         0.606386   
                                                       0.0010         0.633269   
                                          0.5          0.0001         0.633844   
                                                       0.0005         0.609579   
                                                       0.0010         0.640166   
                           (512, 512)     0.1          0.0001         0.626692   
                                                       0.0005         0.622797   
                                                       0.0010         0.623372   
                                          0.5          0.0001         0.631737   
                                                       0.0005         0.612261   
                                                    

In [11]:
best_swnu_network_umap_kfold_20

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.611877,0.487561,"[0.7451507012831992, 0.3990498812351544, 0.318...",0.481610,"[0.7697287299630087, 0.3835616438356164, 0.291...",0.496281,"[0.7220936957779063, 0.4158415841584158, 0.350...",None,0.665382,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.625287,0.495819,"[0.7555684961420875, 0.3927364864864865, 0.339...",0.491809,"[0.7607739665787159, 0.4022491349480969, 0.312...",0.501669,"[0.7504337767495662, 0.38366336633663367, 0.37...",None,0.679994,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.627203,0.507238,"[0.7518998658918193, 0.43647949024293103, 0.33...",0.500449,"[0.7755917614509683, 0.42186297151655117, 0.30...",0.516950,"[0.7296124927703875, 0.4521452145214521, 0.369...",None,0.663295,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [12]:
best_swnu_network_umap_kfold_20["f1"].mean()

0.49687246916673083

In [13]:
best_swnu_network_umap_kfold_20["precision"].mean()

0.4912894996958111

In [14]:
best_swnu_network_umap_kfold_20["recall"].mean()

0.5049665801359392

In [15]:
np.stack(best_swnu_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.75087302, 0.40942195, 0.33032243])

In [16]:
np.stack(best_swnu_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.76869815, 0.40255792, 0.30261243])

In [17]:
np.stack(best_swnu_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.73404666, 0.41721672, 0.36363636])

In [9]:
size = 20
(
    swnu_network_grp_kfold_20,
    best_swnu_network_grp_kfold_20,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    embedding_dim=embedding_dim,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features,
    standardise_method=standardise_method,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_grp_focal_2_20_kfold.csv


In [10]:
swnu_network_grp_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2721244/918308482.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_20.groupby(["dimensions",


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (64, 64)       0.1          0.0001         0.609579   
                                                       0.0005         0.586526   
                                                       0.0010         0.587356   
                                          0.5          0.0001         0.607407   
                                                       0.0005         0.581290   
                                                       0.0010         0.610026   
                           (128, 128)     0.1          0.0001         0.619668   
                                                       0.0005         0.604662   
                                                       0.0010         0.614943   
                                          0.5          0.0001         0.607535   
                                                       0.0005         0.588889   
                                                       0.0010         0.606258   
                           (256, 256)     0.1          0.0001         0.622861   
                                                       0.0005         0.605619   
                                                       0.0010         0.612580   
                                          0.5          0.0001         0.612835   
                                                       0.0005         0.609195   
                                                       0.0010         0.615198   
                           (512, 512)     0.1          0.0001         0.633078   
                                                       0.0005         0.619923   
                                                       0.0010         0.606386   
                                          0.5          0.0001         0.610664   
                                                       0.0005         0.616156   
                                                       0.0010         0.594508   
           (12,)           (64, 64)       0.1          0.0001         0.600894   
                                                       0.0005         0.595594   
                                                       0.0010         0.583844   
                                          0.5          0.0001         0.616922   
                                                       0.0005         0.594444   
                                                       0.0010         0.589400   
                           (128, 128)     0.1          0.0001         0.612516   
                                                       0.0005         0.596232   
                                                       0.0010         0.593870   
                                          0.5          0.0001         0.600447   
                                                       0.0005         0.582695   
                                                       0.0010         0.604023   
                           (256, 256)     0.1          0.0001         0.614687   
                                                       0.0005         0.594700   
                                                       0.0010         0.601277   
                                          0.5          0.0001         0.605492   
                                                       0.0005         0.569987   
                                                       0.0010         0.593359   
                           (512, 512)     0.1          0.0001         0.621264   
                                                       0.0005         0.594955   
                                                       0.0010         0.585632   
                                          0.5          0.0001         0.612324   
                                                       0.0005         0.595402   
                                                    

In [11]:
swnu_network_grp_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2721244/918308482.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_20.groupby(["dimensions",


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (64, 64)       0.1          0.0001         0.609579   
                                                       0.0005         0.586526   
                                                       0.0010         0.587356   
                                          0.5          0.0001         0.607407   
                                                       0.0005         0.581290   
                                                       0.0010         0.610026   
                           (128, 128)     0.1          0.0001         0.619668   
                                                       0.0005         0.604662   
                                                       0.0010         0.614943   
                                          0.5          0.0001         0.607535   
                                                       0.0005         0.588889   
                                                       0.0010         0.606258   
                           (256, 256)     0.1          0.0001         0.622861   
                                                       0.0005         0.605619   
                                                       0.0010         0.612580   
                                          0.5          0.0001         0.612835   
                                                       0.0005         0.609195   
                                                       0.0010         0.615198   
                           (512, 512)     0.1          0.0001         0.633078   
                                                       0.0005         0.619923   
                                                       0.0010         0.606386   
                                          0.5          0.0001         0.610664   
                                                       0.0005         0.616156   
                                                       0.0010         0.594508   
           (12,)           (64, 64)       0.1          0.0001         0.600894   
                                                       0.0005         0.595594   
                                                       0.0010         0.583844   
                                          0.5          0.0001         0.616922   
                                                       0.0005         0.594444   
                                                       0.0010         0.589400   
                           (128, 128)     0.1          0.0001         0.612516   
                                                       0.0005         0.596232   
                                                       0.0010         0.593870   
                                          0.5          0.0001         0.600447   
                                                       0.0005         0.582695   
                                                       0.0010         0.604023   
                           (256, 256)     0.1          0.0001         0.614687   
                                                       0.0005         0.594700   
                                                       0.0010         0.601277   
                                          0.5          0.0001         0.605492   
                                                       0.0005         0.569987   
                                                       0.0010         0.593359   
                           (512, 512)     0.1          0.0001         0.621264   
                                                       0.0005         0.594955   
                                                       0.0010         0.585632   
                                          0.5          0.0001         0.612324   
                                                       0.0005         0.595402   
                                                    

In [12]:
best_swnu_network_grp_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.635057,0.513902,"[0.7599046767947572, 0.44278996865203757, 0.33...",0.508010,"[0.7834766584766585, 0.4216417910447761, 0.318...",0.521900,"[0.7377096587622903, 0.4661716171617162, 0.361...",None,0.672768,...,1,False,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.630268,0.505231,"[0.7581354441512753, 0.4086992203528929, 0.348...",0.500575,"[0.7687277051129607, 0.40653061224489795, 0.32...",0.511089,"[0.7478311162521689, 0.41089108910891087, 0.37...",None,0.681118,...,12,False,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.633908,0.509132,"[0.7600880410858399, 0.42421711899791237, 0.34...",0.503744,"[0.7715221924337206, 0.4294167371090448, 0.310...",0.517255,"[0.7489878542510121, 0.41914191419141916, 0.38...",None,0.678227,...,123,False,focal,2,True,5,Conv1d,None,concatenation,64


In [13]:
best_swnu_network_grp_kfold_20["f1"].mean()

0.5094214141003365

In [14]:
best_swnu_network_grp_kfold_20["precision"].mean()

0.5041095552865794

In [15]:
best_swnu_network_grp_kfold_20["recall"].mean()

0.516748138858613

In [16]:
np.stack(best_swnu_network_grp_kfold_20["f1_scores"]).mean(axis=0)

array([0.75937605, 0.42523544, 0.34365275])

In [17]:
np.stack(best_swnu_network_grp_kfold_20["precision_scores"]).mean(axis=0)

array([0.77457552, 0.41919638, 0.31855677])

In [18]:
np.stack(best_swnu_network_grp_kfold_20["recall_scores"]).mean(axis=0)

array([0.74484288, 0.43206821, 0.37333333])